
# Función para leer el set de datos



In [1]:
import numpy as np
import os
import gzip

def cargar_set(rute, tipo="train"):

  ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz'%tipo)
  ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)

  with gzip.open(ruta_categorias, 'rb') as rut_cat:
    etiquetas = np.frombuffer(rut_cat.read(), dtype = np.uint8, offset = 8)

  with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
    imagenes = np.frombuffer(rut_imgs.read(), dtype = np.uint8, offset= 16).reshape(len(etiquetas), 784)

  return imagenes, etiquetas

# Acceso a la información en Google Drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

ruta = 'gdrive/My Drive/Big_Data_unal/Data_sets/fashion_mnist_data'

X_train, Y_train = cargar_set(ruta, tipo = 'train')
X_test, Y_test = cargar_set(ruta, tipo = 'test')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).



#Reajustar (Reshape) del tamaño de los datos


In [3]:
X_train = X_train[0:59904,:]
X_test = X_test[0:9984,:]
Y_train = Y_train[0:59904]
Y_test = Y_test[0:9984]

X_train = np.reshape(X_train, (59904,28,28,1))
X_test = np.reshape(X_test, (9984,28,28,1))

# Importar TensorFlow 2 con Keras

In [4]:
%tensorflow_version 2.x
import tensorflow as tf

print('Versión de TensorFlow' + tf.__version__)

Versión de TensorFlow2.4.1


# Configurar TPU


In [5]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu = 'grpc://' + os.environ['COLAB_TPU_ADDR'])
  print('TPU encontrada', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('Error: TPU no encontrada')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

TPU encontrada ['10.2.26.226:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.2.26.226:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.2.26.226:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


# Creación del modelo usando TPU strategy


In [6]:
tf.random.set_seed(200)

with tpu_strategy.scope():

  modelo = tf.keras.models.Sequential()

  modelo.add(tf.keras.layers.BatchNormalization(input_shape = X_train.shape[1:]))
  modelo.add(tf.keras.layers.Conv2D(64, (5, 5), padding = 'same', activation ='elu'))
  modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides =(2,2)))
  modelo.add(tf.keras.layers.Dropout(0.25))

  modelo.add(tf.keras.layers.BatchNormalization(input_shape = X_train.shape[1:]))
  modelo.add(tf.keras.layers.Conv2D(128, (5, 5), padding = 'same', activation ='elu'))
  modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides =(2,2)))
  modelo.add(tf.keras.layers.Dropout(0.25))

  modelo.add(tf.keras.layers.BatchNormalization(input_shape = X_train.shape[1:]))
  modelo.add(tf.keras.layers.Conv2D(256, (5, 5), padding = 'same', activation ='elu'))
  modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides =(2,2)))
  modelo.add(tf.keras.layers.Dropout(0.25))

  modelo.add(tf.keras.layers.Flatten())
  modelo.add(tf.keras.layers.Dense(256))
  modelo.add(tf.keras.layers.Activation('elu'))
  modelo.add(tf.keras.layers.Dropout(0.5))
  modelo.add(tf.keras.layers.Dense(10))
  modelo.add(tf.keras.layers.Activation('softmax'))

  # Compilar e imprimir en pantalla la información del modelo

  modelo.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

  modelo.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 64)        1664      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)         0

# Entrenamiento con TPU

In [7]:
x_train = X_train.astype(np.float32)
Y_train = Y_train.astype(np.float32)
x_test = X_test.astype(np.float32)
Y_test = Y_test.astype(np.float32)

In [8]:
import timeit


def entrenamiento_tpu():
  
  modelo.fit(X_train, Y_train, validation_data = (X_test, Y_test), batch_size = 128, epochs = 2, verbose = 1)

  return None


tpu_time = timeit.timeit('entrenamiento_tpu()', number = 1, setup = 'from __main__ import entrenamiento_tpu')

Epoch 1/2
468/468 [==============================] - 14s 20ms/step - loss: 1.0756 - accuracy: 0.7006 - val_loss: 0.4072 - val_accuracy: 0.8603
Epoch 2/2
468/468 [==============================] - 8s 17ms/step - loss: 0.4274 - accuracy: 0.8487 - val_loss: 0.3333 - val_accuracy: 0.8890


In [9]:
print('El tiempo de entrenamiento en TPU es de: ' + str(tpu_time/60) + " minutos")

El tiempo de entrenamiento en TPU es de: 0.3868405437666676 minutos
